In [1]:
crap version

import scipy.io 
import numpy as np
from numpy import linalg as LA
import matplotlib
import time
from PIL import Image

function for viewing images

In [2]:
def show_img(img):
    # shows image as png
    # img must be a 2576 1d vector
    new_img = img.copy()
    new_img.resize((46,56))
    im = Image.fromarray(np.transpose(new_img))
    im.show()

Load the relevant data into .mat into a python dictionary

In [3]:
mat = scipy.io.loadmat('face.mat')
raw_data = np.transpose(mat['X'])

In [4]:
print(raw_data.shape)

(520, 2576)


init arrays for training and test data

In [5]:
training_data = []
# np.empty([int(520*0.8), 2576])
testing_data = []
# np.empty([int(520*0.2), 2576])

create training and test data

In [6]:
for x in range(52):
        # 8/2 ratio for training and testing datasets	
	training_data[x*8:(x+1)*8] = raw_data[x*10:x*10+8]
	testing_data[x*2:(x+1)*2] = raw_data[x*10+8:(x+1)*10]
training_data = np.array(training_data).copy()
testing_data = np.array(testing_data).copy()

In [7]:
# show_img(training_data[0])
print(len(training_data))

416


Get deviation from the mean face

In [8]:
mean_train_face = np.mean(training_data, axis=0)
mean_test_face = np.mean(testing_data, axis=0)
diff_train_face = training_data - mean_train_face
diff_test_face = testing_data - mean_test_face

In [9]:
print(mean_train_face.shape)

(2576,)


In [10]:
show_img(mean_train_face)

compute the covariance matrix for low-dimensional computation

In [11]:
start = time.time()
low_cov_mat = (1/len(raw_data))*np.dot(training_data,training_data.T)
end = time.time()
print("time required to compute low-dim-cov-mat : ",end - start)
print("shape of cov_mat: ", low_cov_mat.shape)

time required to compute low-dim-cov-mat :  1.413844347000122
shape of cov_mat:  (416, 416)


compute the eigenvalues and eigenvectors of the covariance matrix
w -> eigenvalues
v -> eigenvectors

In [12]:
start = time.time()
low_w, low_v = LA.eig(low_cov_mat)
# reconstruct original eignevectors
# do we need to normalise
low_computed_u = np.matmul(training_data.T,low_v).T # is probably incredibly wrong
end = time.time()
print("time required to compute eigenvalues and eigenvectors of low-dim-cov-mat : ",end-start)

time required to compute eigenvalues and eigenvectors of low-dim-cov-mat :  2.2372586727142334


In [13]:
print(low_computed_u.shape)

(416, 2576)


normalise eigen vectors for printing

In [14]:
# for x in nz_eigenvectors:
#     x /= LA.norm(nz_eigenvectors, ord=np.inf, axis=0)
# nz_eigenvectors *= 128
# nz_eigenvectors += 128

In [15]:
# print(nz_eigenvectors)

normalise for eigen vec for finding weights

In [16]:
for x in low_computed_u:
    x /= LA.norm(x, ord=2, axis=0)

In [17]:
print(LA.norm(low_computed_u[0], ord=2, axis=0))

0.9999999999999999


In [18]:
# print(LA.norm(nz_eigenvectors, ord=2, axis=1))

Calculate weights associated with each eigenvec

In [19]:
lowdim_eigenvec = low_computed_u[0:100]

In [20]:
print(lowdim_eigenvec.shape)

(100, 2576)


In [21]:
weights = np.matmul(lowdim_eigenvec, training_data.T).T

In [22]:
print(weights.shape)

(416, 100)


In [23]:
print(training_data[0])
print(low_computed_u.shape)
print(training_data.shape)
print(low_computed_u[1])
print(training_data[0])

[106 103 103 ...  37  36  41]
(416, 2576)
(416, 2576)
[0.01575876 0.0223506  0.02546815 ... 0.01207801 0.01735189 0.01721741]
[106 103 103 ...  37  36  41]


In [24]:
lowdim_weights = weights

In [25]:
print(lowdim_weights.shape)

(416, 100)


In [26]:
print(low_computed_u.shape)

(416, 2576)


Reconstruct differences from mean

In [27]:
crappyfaces = np.matmul(lowdim_eigenvec.T, lowdim_weights.T).T

In [28]:
print(crappyfaces.shape)

(416, 2576)


In [29]:
print(crappyfaces[0].shape)

(2576,)


In [30]:
for x in crappyfaces[0:15]:
    show_img(x)
for x in training_data[0:15]:
    show_img(x)

In [31]:
for x in crappyfaces[0:15]:
    show_img(x + mean_test_face)

In [32]:
# for x in nz_eigenvectors[0:20]:
#     show_img(x)

sort form largest to smallest eigen vector

In [33]:
low_eigenval_vec = [(x,y) for x, y in zip(low_w, low_computed_u)]
sorted_low = sorted(low_eigenval_vec, key=lambda eigenv: eigenv[0], reverse=True)
# print(sorted_low)

compute the covariance matrix for normal PCA computation

In [34]:
normal_cov_mat = np.cov(np.transpose(dif_train_face)) # need timing
print("Length of normal cov matrix", len(normal_cov_mat))

NameError: name 'dif_train_face' is not defined

compute eigenvalues and vectors for normal PCA computation
normal_t -> eigenvalues
normal_u -> eigenvectors

In [ ]:
normal_t, normal_u = LA.eig(normal_cov_mat) # need to do timing

get the eigen vectors corresponding to the largest 416 eigen values

In [ ]:
normal_eigenval_vec = [(x,y) for x, y in zip(normal_t, normal_u)]
sorted_normal = sorted(normal_eigenval_vec, key=lambda eigenv: eigenv[0], reverse=True)
sorted_normal = sorted_normal[0:416]

check that the eigen values and the eigen vectors are the same from both computations

In [ ]:
print("are the computed eigen values and vectors the same?")
print("size of sorted", len)
if np.array_equal(sorted_normal, sorted_low):
	print("yoyoyo")
else:
	print("f off")